## initialize

In [1]:
%cd ..

/Users/ales/dev/repos/ai-audio-books


/Users/ales/dev/python-venvs/ai-audio-books/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%load_ext autoreload
%autoreload 2

In [23]:
import os

from pprint import pprint

import dotenv
import pandas as pd
from httpx import Timeout
from pydantic import BaseModel
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback

from IPython.display import Audio

import data.samples_to_split as samples

from src.lc_callbacks import LCMessageLoggerAsync
from src.schemas import AudioOutputFormat, TTSParams, TTSTimestampsResponse, TTSTimestampsAlignment
from src.text_split_chain import create_split_text_chain
from src import tts
from src.utils import GPTModels

In [4]:
dotenv.load_dotenv()

True

In [5]:
text = samples.GATSBY_2
# text = """\
# Margaret: hello, how are you Tom?
# Tom: nice, thanks. And you?
# """

## split text into character phrases

In [6]:
chain = create_split_text_chain(llm_model=GPTModels.GPT_4o)
# chain = create_split_text_chain(llm_model=GPTModels.GPT_4_TURBO_2024_04_09)
with get_openai_callback() as cb:
    res = chain.invoke({"text": text}, config={"callbacks": [LCMessageLoggerAsync()]})

2024-11-03 17:49:47,976 [INFO] audio-books (lc_callbacks.py): call to gpt-4o with 2 messages:
{'role': 'system', 'content': 'you are provided with the book sample.\nplease rewrite it and insert xml tags indicating character to whom current phrase belongs.\nfor example: <narrator>I looked at her</narrator><Jill>What are you looking at?</Jill>\n\nNotes:\n- sometimes narrator is one of characters taking part in the action.\nin this case use narrator\'s name (if available) instead of "narrator"\n- if it\'s impossible to identify character name from the text provided, use codes "c1", "c2", etc,\nwhere "c" prefix means character and number is used to enumerate unknown characters\n- all quotes of direct speech must be attributed to characters, for example:\n<Tom>“She’s a nice girl,”</Tom><narrator>said Tom after a moment.</narrator>\nmind that sometimes narrator could also be a character.\n- use ALL available context to determine the character.\nsometimes the character name becomes clear from

In [7]:
res.characters

['Jordan', 'Tom', 'Daisy', 'narrator']

In [8]:
print(res.text_annotated)

<narrator>Inside, the crimson room bloomed with light. Tom and Miss Baker sat at either end of the long couch and she read aloud to him from the Saturday Evening Post—the words, murmurous and uninflected, running together in a soothing tune. The lamplight, bright on his boots and dull on the autumn-leaf yellow of her hair, glinted along the paper as she turned a page with a flutter of slender muscles in her arms.</narrator>

<narrator>When we came in she held us silent for a moment with a lifted hand.</narrator>

<Jordan>“To be continued,”</Jordan> <narrator>she said, tossing the magazine on the table,</narrator> <Jordan>“in our very next issue.”</Jordan>

<narrator>Her body asserted itself with a restless movement of her knee, and she stood up.</narrator>

<Jordan>“Ten o’clock,”</Jordan> <narrator>she remarked, apparently finding the time on the ceiling.</narrator> <Jordan>“Time for this good girl to go to bed.”</Jordan>

<Daisy>“Jordan’s going to play in the tournament tomorrow,”</Da

In [9]:
res.phrases

[CharacterPhrase(character='narrator', text='Inside, the crimson room bloomed with light. Tom and Miss Baker sat at either end of the long couch and she read aloud to him from the Saturday Evening Post—the words, murmurous and uninflected, running together in a soothing tune. The lamplight, bright on his boots and dull on the autumn-leaf yellow of her hair, glinted along the paper as she turned a page with a flutter of slender muscles in her arms.'),
 CharacterPhrase(character='narrator', text='When we came in she held us silent for a moment with a lifted hand.'),
 CharacterPhrase(character='Jordan', text='“To be continued,”'),
 CharacterPhrase(character='narrator', text='she said, tossing the magazine on the table,'),
 CharacterPhrase(character='Jordan', text='“in our very next issue.”'),
 CharacterPhrase(character='narrator', text='Her body asserted itself with a restless movement of her knee, and she stood up.'),
 CharacterPhrase(character='Jordan', text='“Ten o’clock,”'),
 Characte

In [10]:
print(res.text_raw)

Inside, the crimson room bloomed with light. Tom and Miss Baker sat at
either end of the long couch and she read aloud to him from the
Saturday Evening Post—the words, murmurous and uninflected, running
together in a soothing tune. The lamplight, bright on his boots and
dull on the autumn-leaf yellow of her hair, glinted along the paper as
she turned a page with a flutter of slender muscles in her arms.

When we came in she held us silent for a moment with a lifted hand.

“To be continued,” she said, tossing the magazine on the table, “in
our very next issue.”

Her body asserted itself with a restless movement of her knee, and she
stood up.

“Ten o’clock,” she remarked, apparently finding the time on the
ceiling. “Time for this good girl to go to bed.”

“Jordan’s going to play in the tournament tomorrow,” explained Daisy,
“over at Westchester.”

“Oh—you’re Jordan Baker.”

I knew now why her face was familiar—its pleasing contemptuous
expression had looked out at me from many rotogravur

In [11]:
print(res.to_pretty_text())

characters: ['Jordan', 'Tom', 'Daisy', 'narrator']
--------------------
[narrator] Inside, the crimson room bloomed with light. Tom and Miss Baker sat at either end of the long couch and she read aloud to him from the Saturday Evening Post—the words, murmurous and uninflected, running together in a soothing tune. The lamplight, bright on his boots and dull on the autumn-leaf yellow of her hair, glinted along the paper as she turned a page with a flutter of slender muscles in her arms.
[narrator] When we came in she held us silent for a moment with a lifted hand.
[Jordan] “To be continued,”
[narrator] she said, tossing the magazine on the table,
[Jordan] “in our very next issue.”
[narrator] Her body asserted itself with a restless movement of her knee, and she stood up.
[Jordan] “Ten o’clock,”
[narrator] she remarked, apparently finding the time on the ceiling.
[Jordan] “Time for this good girl to go to bed.”
[Daisy] “Jordan’s going to play in the tournament tomorrow,”
[narrator] explai

In [12]:
print(f'LLM usage:\n\n{cb}')

LLM usage:

Tokens Used: 1816
	Prompt Tokens: 877
	Completion Tokens: 939
Successful Requests: 1
Total Cost (USD): $0.011582499999999999


## map characters to voices

In [13]:
from src.select_voice_chain import VoiceSelector

In [14]:
vs = VoiceSelector()

2024-11-03 17:50:03,555 [INFO] audio-books (select_voice_chain.py): reading voice data from: "data/11labs_available_tts_voices.reviewed.csv"
2024-11-03 17:50:03,560 [INFO] audio-books (select_voice_chain.py): df.shape=(34, 15)
2024-11-03 17:50:03,562 [INFO] audio-books (select_voice_chain.py): filtering df by "manual_quality_review" column
2024-11-03 17:50:03,566 [INFO] audio-books (select_voice_chain.py): df.shape after filtering voices: (25, 15)


In [15]:
chain = vs.create_voice_mapping_chain(llm_model=GPTModels.GPT_4o)

In [16]:
chain

RunnableAssign(mapper={
  charater_props: ChatPromptTemplate(input_variables=['characters', 'text'], input_types={}, partial_variables={'available_genders': '"male", "female"', 'available_age_groups': '"middle_aged", "young", "old"', 'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"CharacterProperties": {"properties": {"gender": {"title": "Gender", "type": "string"}, "age_group": {"title": "Age Group", "type": "string"}}, "required": ["gender", "age_group"], "title": "CharacterProperties", "type": "object"}}, "properties": {"character2props": {"addi

In [17]:
res2 = chain.invoke(
    {"text": res.text_annotated, "characters": res.characters},
    config={"callbacks": [LCMessageLoggerAsync()]},
)

2024-11-03 17:50:03,650 [INFO] audio-books (lc_callbacks.py): call to gpt-4o with 2 messages:
{'role': 'system', 'content': 'You are a helpful assistant proficient in literature and psychology.\nOur goal is to create an audio book from the given text.\nFor that we need to hire voice actors.\nPlease help us to find the right actor for each character present in the text.\n\nYou are provided with the text split by the characters\nto whom text parts belong to.\n\nYour task is to assign available properties to each character provided.\nList of available properties:\n- gender: "male", "female"\n- age_group: "middle_aged", "young", "old"\n\nNOTES:\n- assign EXACTLY ONE property value for each property\n- select properties values ONLY from the list of AVAILABLE property values\n- fill properties for ALL characters from the list provided\n- DO NOT include any characters absent in the list provided\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs

In [18]:
res2

SelectVoiceChainOutput(character2props={'Jordan': CharacterPropertiesNullable(gender='female', age_group='young'), 'Tom': CharacterPropertiesNullable(gender='male', age_group='middle_aged'), 'Daisy': CharacterPropertiesNullable(gender='female', age_group='young'), 'narrator': CharacterPropertiesNullable(gender='male', age_group='young')}, character2voice={'Jordan': '1btZhL2wthuOhUqvI0bB', 'Daisy': 'K5DRk4s8l1HFKsggS25u', 'Tom': 'RPEIZnKMqlQiZyZd1Dae', 'narrator': 'xSI29a9HAKdsWv3idXSN'})

In [19]:
character2voice = res2.character2voice
character2voice

{'Jordan': '1btZhL2wthuOhUqvI0bB',
 'Daisy': 'K5DRk4s8l1HFKsggS25u',
 'Tom': 'RPEIZnKMqlQiZyZd1Dae',
 'narrator': 'xSI29a9HAKdsWv3idXSN'}

## generate audio

In [20]:
res.phrases

[CharacterPhrase(character='narrator', text='Inside, the crimson room bloomed with light. Tom and Miss Baker sat at either end of the long couch and she read aloud to him from the Saturday Evening Post—the words, murmurous and uninflected, running together in a soothing tune. The lamplight, bright on his boots and dull on the autumn-leaf yellow of her hair, glinted along the paper as she turned a page with a flutter of slender muscles in her arms.'),
 CharacterPhrase(character='narrator', text='When we came in she held us silent for a moment with a lifted hand.'),
 CharacterPhrase(character='Jordan', text='“To be continued,”'),
 CharacterPhrase(character='narrator', text='she said, tossing the magazine on the table,'),
 CharacterPhrase(character='Jordan', text='“in our very next issue.”'),
 CharacterPhrase(character='narrator', text='Her body asserted itself with a restless movement of her knee, and she stood up.'),
 CharacterPhrase(character='Jordan', text='“Ten o’clock,”'),
 Characte

In [21]:
tts_responses = []

for phrase in res.phrases:
    voice_id = character2voice[phrase.character]
    tts_params = TTSParams(voice_id=voice_id, text=phrase.text)
    response = await tts.tts_w_timestamps(tts_params)
    tts_responses.append(response)

2024-11-03 17:50:05,529 [INFO] audio-books (tts.py): request to 11labs TTS endpoint with params {'voice_id': 'xSI29a9HAKdsWv3idXSN', 'output_format': <AudioOutputFormat.MP3_44100_192: 'mp3_44100_192'>} for the following text: "Inside, the crimson room bloomed with light. Tom and Miss Baker sat at either end of the long couch and she read aloud to him from the Saturday Evening Post—the words, murmurous and uninflected, running together in a soothing tune. The lamplight, bright on his boots and dull on the autumn-leaf yellow of her hair, glinted along the paper as she turned a page with a flutter of slender muscles in her arms."
2024-11-03 17:50:13,974 [INFO] httpx (_client.py): HTTP Request: POST https://api.elevenlabs.io/v1/text-to-speech/xSI29a9HAKdsWv3idXSN/with-timestamps?output_format=mp3_44100_192 "HTTP/1.1 200 OK"
2024-11-03 17:50:14,120 [INFO] audio-books (tts.py): request to 11labs TTS endpoint with params {'voice_id': 'xSI29a9HAKdsWv3idXSN', 'output_format': <AudioOutputFormat

In [22]:
alignments = [response.alignment for response in tts_responses]
char2time = TTSTimestampsAlignemnt.combine_alignments(alignments=alignments)
char2time.to_dataframe()

,char,start,end
0,I,0.000,0.058
1,n,0.058,0.174
2,s,0.174,0.255
3,i,0.255,0.406
4,d,0.406,0.453
...,...,...,...
2532,d,147.860,147.884
2533,,147.884,147.918
2534,m,147.918,147.953
2535,e,147.953,148.035


In [23]:
print(f'len of original text: {len(text)}')
# add 1 extra char imitating space between character phrases
joined_phrases = ' '.join([x.text for x in res.phrases])
print(f'len of joined character phrases: {len(joined_phrases)}')
print(f'len of char2time mapping: {len(char2time.characters)}')

len of original text: 2560
len of joined character phrases: 2537
len of char2time mapping: 2537


## design effects

In [24]:
from src.sound_effects_design import create_sound_effects_design_chain, SoundEffectDescription

In [25]:
chain = create_sound_effects_design_chain(llm_model=GPTModels.GPT_4o)

with get_openai_callback() as cb:
    res = chain.invoke(
        {"text": text}, config={"callbacks": [LCMessageLoggerAsync()]}
    )

2024-11-03 17:50:54,763 [INFO] audio-books (lc_callbacks.py): call to gpt-4o with 2 messages:
{'role': 'system', 'content': 'You are an expert in directing audiobooks creation.\nYour task is to design sound effects layed over the audio book.\nYou are provided with the audiobook text chunk -\ninsert XML tags describing sound effects, their place and duration.\n\nXML effect tags must have following structure:\n<effect prompt="prompt to be passed to text-to-sound-effect AI model">original line from the text</effect>\n\nAdditional requirements:\n- In the very beginning, analyze the whole text chunk provided in order to understand events and atmosphere.\n- Prompts you place inside XML tags are going to be passes to text-to-sound-effect AI model.\nThus, it\'s required to write prompts rich in details.\n- Do not generate long-running background or ambient music, crowd talking\n- Aim for episodical sound effects, highlighting atmosphere and characters\' actions.\nFor example, cracking of stair

In [26]:
# print(res.text_raw)
# print(res.text_annotated)

In [27]:
res.sound_effects_descriptions

[SoundEffectDescription(prompt='room filled with warm, inviting light', text_between_tags='crimson room bloomed with light', ix_start_llm_response=12, ix_end_llm_response=107, ix_start_orig_text=12, ix_end_orig_text=43, duration=-1),
 SoundEffectDescription(prompt='soft rustling of paper as a page is turned', text_between_tags='glinted along the paper as she turned a page with a flutter of slender muscles in her arms', ix_start_llm_response=379, ix_end_llm_response=538, ix_start_orig_text=315, ix_end_orig_text=405, duration=-1),
 SoundEffectDescription(prompt='gentle, commanding gesture', text_between_tags='lifted hand', ix_start_llm_response=596, ix_end_llm_response=660, ix_start_orig_text=463, ix_end_orig_text=474, duration=-1),
 SoundEffectDescription(prompt='magazine landing softly on a table', text_between_tags='tossing the magazine on the table', ix_start_llm_response=692, ix_end_llm_response=786, ix_start_orig_text=506, ix_end_orig_text=539, duration=-1),
 SoundEffectDescription

In [28]:
sed = res.sound_effects_descriptions[0]
print(f'{sed.duration=}')

sed.duration=-1


In [37]:
# sound_effects_descriptions = res.sound_effects_descriptions

In [38]:
# # TODO: move to function
# for sed in sound_effects_descriptions:
#     ix_start, ix_end = sed.ix_start_llm_response, sed.ix_end_llm_response
#     time_start = char2time.get_start_time_by_char_ix(ix_start)
#     time_end = char2time.get_end_time_by_char_ix(ix_end)
#     duration = time_end - time_start
#     sed.duration = duration

In [39]:
from src.config import logger

In [42]:
# # filter
# min_sound_effect_durations = 1
# sound_effects_descriptions_filtered = [
#     sed for sed in sound_effects_descriptions if sed.duration > min_sound_effect_durations
# ]
# len_orig = len(sound_effects_descriptions)
# len_new = len(sound_effects_descriptions_filtered)
# logger.info(
#     f'{len_new} out of {len_orig} original sound effects are kept after filtering by min duration: '
#     f'{min_sound_effect_durations}'
# )

2024-11-03 17:57:44,223 [INFO] audio-books (789017501.py): 8 out of 9 original sound effects are kept after filtering by min duration: 1


In [43]:
for sed in sound_effects_descriptions_filtered:
    text_bw_tags = sed.text_between_tags
    orig_text_from_calc_indices = text[sed.ix_start_orig_text : sed.ix_end_orig_text]
    print(f'{sed.prompt=}')
    print(f'{text_bw_tags=}')
    print(f'{orig_text_from_calc_indices=}')
    print(f'{orig_text_from_calc_indices == text_bw_tags=}')
    print(f'')

sed.prompt='room filled with warm, inviting light'
text_bw_tags='crimson room bloomed with light'
orig_text_from_calc_indices='crimson room bloomed with light'
orig_text_from_calc_indices == text_bw_tags=True

sed.prompt='soft rustling of paper as a page is turned'
text_bw_tags='glinted along the paper as she turned a page with a flutter of slender muscles in her arms'
orig_text_from_calc_indices='glinted along the paper as\nshe turned a page with a flutter of slender muscles in her arms'
orig_text_from_calc_indices == text_bw_tags=False

sed.prompt='gentle, commanding gesture'
text_bw_tags='lifted hand'
orig_text_from_calc_indices='lifted hand'
orig_text_from_calc_indices == text_bw_tags=True

sed.prompt='magazine landing softly on a table'
text_bw_tags='tossing the magazine on the table'
orig_text_from_calc_indices='tossing the magazine on the table'
orig_text_from_calc_indices == text_bw_tags=True

sed.prompt='restless movement, fabric rustling'
text_bw_tags='restless movement of he

## generate effects

In [45]:
from src.schemas import SoundEffectsParams

In [46]:
# sed = sound_effects_descriptions_filtered[0]
# sed

SoundEffectDescription(prompt='room filled with warm, inviting light', text_between_tags='crimson room bloomed with light', ix_start_llm_response=12, ix_end_llm_response=107, ix_start_orig_text=12, ix_end_orig_text=43, duration=5.271)

In [47]:
# params = SoundEffectsParams(
#     text=sed.prompt,
#     duration_seconds=sed.duration,
#     prompt_influence=0.75  # NOTE: default, seems to work fine
# )

In [53]:
sound_effect_response = await tts.sound_generation_consumed(params=params)

2024-11-03 18:14:57,872 [INFO] audio-books (tts.py): request to 11labs sound effect generation with params {'duration_seconds': 5.271, 'prompt_influence': 0.75} for the following text: "room filled with warm, inviting light"


2024-11-03 18:15:03,710 [INFO] httpx (_client.py): HTTP Request: POST https://api.elevenlabs.io/v1/sound-generation "HTTP/1.1 200 OK"


In [63]:
type(sound_effect_response[0])

bytes

In [66]:
from src import utils

In [67]:
# effects_dp = os.path.join('data', 'tmp', 'sound_effects')
# os.makedirs(effects_dp, exist_ok=True)

In [68]:
# out_fp = os.path.join(effects_dp, 'sounf_effect.wav')
# utils.write_chunked_bytes(data=sound_effect_response, fp=out_fp)

2024-11-03 18:25:48,886 [INFO] audio-books (utils.py): saving to: "data/tmp/sound_effects/sounf_effect.wav"


## generate audio with timestamps

In [5]:
# text = samples.ARCH_WIKI_1[:200]
# pprint(text)

In [6]:
# text = '''\
# hello, this is the test when I am voicing 123 different phrases (some in parentheses),
# with newlines
# some unreadable characters: #!@%*&
# LooLLL123
# how is it??? going!!
# and some smiles: :))
# '''

In [7]:
text = '<speak>\n    <p>\n        <s>\n            Inside, the <emphasis level=\"strong\">crimson room</emphasis> bloomed with light.\n        </s>\n        <s>\n            Tom and Miss Baker sat at either end of the long couch, and she <prosody rate=\"slow\" volume=\"soft\">read aloud to him</prosody> from the Saturday Evening Post—\n            the words, <prosody rate=\"medium\" pitch=\"-1st\">murmurous and uninflected,</prosody> <prosody rate=\"slow\" volume=\"soft\">running together in a soothing tune.</prosody>\n        </s>\n        <s>\n            The lamplight, bright on his boots and <prosody volume=\"medium\" pitch=\"+1st\">dull on the autumn-leaf yellow of her hair,</prosody> \n            <prosody rate=\"medium\">glinted along the paper</prosody> as she turned a page with a <prosody rate=\"fast\">flutter</prosody> of slender muscles in her arms.\n        </s>\n    </p>\n</speak>'
print(text)

<speak>
    <p>
        <s>
            Inside, the <emphasis level="strong">crimson room</emphasis> bloomed with light.
        </s>
        <s>
            Tom and Miss Baker sat at either end of the long couch, and she <prosody rate="slow" volume="soft">read aloud to him</prosody> from the Saturday Evening Post—
            the words, <prosody rate="medium" pitch="-1st">murmurous and uninflected,</prosody> <prosody rate="slow" volume="soft">running together in a soothing tune.</prosody>
        </s>
        <s>
            The lamplight, bright on his boots and <prosody volume="medium" pitch="+1st">dull on the autumn-leaf yellow of her hair,</prosody> 
            <prosody rate="medium">glinted along the paper</prosody> as she turned a page with a <prosody rate="fast">flutter</prosody> of slender muscles in her arms.
        </s>
    </p>
</speak>


In [5]:
out_dp = os.path.join('data', 'intonation_experiment', '1')
os.makedirs(out_dp, exist_ok=True)

In [9]:
text = 'Inside, the crimson room bloomed with light.'
params = TTSParams(voice_id="8opUN7sGOKbyojnjvNdl", text=text, seed=672)

for ix in range(3):
    out_fp = os.path.join(out_dp, f'no_ssml.ix{ix}')
    response = await tts.tts_w_timestamps(params)
    response.write_audio_to_file(out_fp, params.output_format)

2024-11-05 01:19:44,111 [INFO] audio-books (tts.py): request to 11labs TTS endpoint with params {'voice_id': '8opUN7sGOKbyojnjvNdl', 'output_format': <AudioOutputFormat.MP3_44100_192: 'mp3_44100_192'>, 'seed': 672} for the following text: "Inside, the crimson room bloomed with light."
2024-11-05 01:19:45,277 [INFO] httpx (_client.py): HTTP Request: POST https://api.elevenlabs.io/v1/text-to-speech/8opUN7sGOKbyojnjvNdl/with-timestamps?output_format=mp3_44100_192 "HTTP/1.1 200 OK"
2024-11-05 01:19:45,344 [INFO] audio-books (utils.py): saving to: "data/intonation_experiment/1/no_ssml.ix0.mp3"
2024-11-05 01:19:45,349 [INFO] audio-books (tts.py): request to 11labs TTS endpoint with params {'voice_id': '8opUN7sGOKbyojnjvNdl', 'output_format': <AudioOutputFormat.MP3_44100_192: 'mp3_44100_192'>, 'seed': 672} for the following text: "Inside, the crimson room bloomed with light."
2024-11-05 01:19:46,211 [INFO] httpx (_client.py): HTTP Request: POST https://api.elevenlabs.io/v1/text-to-speech/8opU

In [10]:
text = 'Inside, the <emphasis level="strong">crimson room</emphasis> bloomed with light.'
params = TTSParams(voice_id="8opUN7sGOKbyojnjvNdl", text=text, seed=672)

for ix in range(3):
    out_fp = os.path.join(out_dp, f'with_ssml.ix{ix}')
    response = await tts.tts_w_timestamps(params)
    response.write_audio_to_file(out_fp, params.output_format)

2024-11-05 01:20:43,720 [INFO] audio-books (tts.py): request to 11labs TTS endpoint with params {'voice_id': '8opUN7sGOKbyojnjvNdl', 'output_format': <AudioOutputFormat.MP3_44100_192: 'mp3_44100_192'>, 'seed': 672} for the following text: "Inside, the <emphasis level="strong">crimson room</emphasis> bloomed with light."
2024-11-05 01:20:44,585 [INFO] httpx (_client.py): HTTP Request: POST https://api.elevenlabs.io/v1/text-to-speech/8opUN7sGOKbyojnjvNdl/with-timestamps?output_format=mp3_44100_192 "HTTP/1.1 200 OK"
2024-11-05 01:20:44,629 [INFO] audio-books (utils.py): saving to: "data/intonation_experiment/1/with_ssml.ix0.mp3"
2024-11-05 01:20:44,630 [INFO] audio-books (tts.py): request to 11labs TTS endpoint with params {'voice_id': '8opUN7sGOKbyojnjvNdl', 'output_format': <AudioOutputFormat.MP3_44100_192: 'mp3_44100_192'>, 'seed': 672} for the following text: "Inside, the <emphasis level="strong">crimson room</emphasis> bloomed with light."
2024-11-05 01:20:45,461 [INFO] httpx (_clien

In [6]:
text = '''
Inside, the crimson room bloomed with light.
Tom and Miss Baker sat at either end of the long couch,
<very-slow> and she read aloud to him from the Saturday Evening Post</very-slow>
- the words, murmurous and uninflected
'''

params = TTSParams(voice_id="8opUN7sGOKbyojnjvNdl", text=text, seed=672)

for ix in range(1):
    out_fp = os.path.join(out_dp, f'with_prosody_rate.ix{ix}')
    response = await tts.tts_w_timestamps(params)
    response.write_audio_to_file(out_fp, params.output_format)

2024-11-05 01:32:44,099 [INFO] audio-books (tts.py): request to 11labs TTS endpoint with params {'voice_id': '8opUN7sGOKbyojnjvNdl', 'output_format': <AudioOutputFormat.MP3_44100_192: 'mp3_44100_192'>, 'seed': 672} for the following text: "
Inside, the crimson room bloomed with light.
Tom and Miss Baker sat at either end of the long couch,
<very-slow> and she read aloud to him from the Saturday Evening Post</very-slow>
- the words, murmurous and uninflected
"
2024-11-05 01:32:46,529 [INFO] httpx (_client.py): HTTP Request: POST https://api.elevenlabs.io/v1/text-to-speech/8opUN7sGOKbyojnjvNdl/with-timestamps?output_format=mp3_44100_192 "HTTP/1.1 200 OK"
2024-11-05 01:32:46,612 [INFO] audio-books (utils.py): saving to: "data/intonation_experiment/1/with_prosody_rate.ix0.mp3"


In [64]:
response_raw = await tts.ELEVEN_CLIENT_ASYNC.text_to_speech.convert_with_timestamps(
    **params.to_dict()
)

/Users/ales/dev/python-venvs/ai-audio-books/lib/python3.12/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `VoiceSettings` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `int` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-10-27 21:44:22,111 [INFO] httpx (_client.py): HTTP Request: POST https://api.elevenlabs.io/v1/text-to-speech/8opUN7sGOKbyojnjvNdl/with-timestamps?output_format=mp3_44100_192 "HTTP/1.1 200 OK"


In [75]:
response_parsed = TTSTimestampsResponse.model_validate(response_raw)

In [86]:
t = response_parsed.alignment.text_joined
t

'hello, this is the test when I am voicing 123 different phrases (some in parentheses),\nwith newlines\nsome unreadable characters: #!@%*&\nLooLLL123\nhow is it??? going!!\nand some smiles: :))\n'

In [89]:
from src.schemas import TTSTimestampsAlignemnt

In [94]:
a1 = TTSTimestampsAlignemnt(
    characters=list('abc'),
    character_start_times_seconds=[0.1, 0.2, 0.3],
    character_end_times_seconds=[0.15, 0.25, 0.35],
)
a2 = TTSTimestampsAlignemnt(
    characters=list('def'),
    character_start_times_seconds=[0.1, 0.2, 0.3],
    character_end_times_seconds=[0.15, 0.25, 0.35],
)
a3 = TTSTimestampsAlignemnt(
    characters=list("ghi"),
    character_start_times_seconds=[0.1, 0.2, 0.3],
    character_end_times_seconds=[0.15, 0.25, 0.35],
)

In [108]:
a = TTSTimestampsAlignemnt.combine_alignments(alignments=[a1, a2, a3])

In [109]:
a.to_dataframe()

,char,start,end
0,a,0.10,0.15
1,b,0.20,0.25
2,c,0.30,0.35
3,d,0.45,0.50
4,e,0.55,0.60
5,f,0.65,0.70
6,g,0.80,0.85
7,h,0.90,0.95
8,i,1.00,1.05


In [110]:
a.get_start_time_by_char_ix(3)

0.44999999999999996

In [111]:
a.get_end_time_by_char_ix(6)

0.85

## compare audio quality for different formats

In [5]:
text = samples.ARCH_WIKI_1[:200]
pprint(text)

('This document is an annotated index of popular articles and important '
 'information for improving and adding functionalities to the installed Arch '
 'system. Readers are assumed to have read and followed t')


In [12]:
params_base = TTSParams(
    voice_id="8opUN7sGOKbyojnjvNdl",
    text="hello, how are you doing? this is the test aiming to decide which audio quality option to use",
    # text=text,
)

In [13]:
# out_dp = "data/compare_audio_quality2"
# os.makedirs(out_dp, exist_ok=True)

# for audio_format in AudioOutputFormat:
#     if audio_format is AudioOutputFormat.ULAW_8000:
#         continue

#     params = params_base.model_copy(deep=True)
#     params.output_format = audio_format

#     response_raw = await ELEVEN_CLIENT_ASYNC.text_to_speech.convert_with_timestamps(
#         **params.to_dict()
#     )
#     response_parsed = TTSTimestampsResponse.model_validate(response_raw)

#     filepath_no_ext = os.path.join(out_dp, f"compare.{audio_format}")
#     out_fp = response_parsed.write_audio_to_file(
#         filepath_no_ext=filepath_no_ext, audio_format=audio_format
#     )

/Users/ales/dev/python-venvs/ai-audio-books/lib/python3.12/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `VoiceSettings` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `int` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-10-27 18:16:54,779 [INFO] httpx (_client.py): HTTP Request: POST https://api.elevenlabs.io/v1/text-to-speech/8opUN7sGOKbyojnjvNdl/with-timestamps?output_format=mp3_22050_32 "HTTP/1.1 200 OK"
2024-10

In [11]:
!ffprobe data/compare_audio_quality/compare.mp3_44100_64.mp3

ffprobe version 7.0.1 Copyright (c) 2007-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex -

## normalize audio

In [20]:
import pydub
import pydub.effects

In [12]:
fp_in = 'data/tmp/tmp.mp3'

In [17]:
audio = pydub.AudioSegment.from_file(fp_in)

In [18]:
audio

In [19]:
audio - 5

In [21]:
pydub.effects.normalize(audio)

In [25]:
from src import utils

In [33]:
utils.normalize_audio(audio, target_dBFS=-20)

In [32]:
utils.normalize_audio(audio, target_dBFS=-25)